In [23]:
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt

In [24]:
con = sqlite3.connect('db.sqlite3')
df = pd.read_sql_query("SELECT * FROM sqlite_master", con)

In [25]:
df

,type,name,tbl_name,rootpage,sql
0,table,django_migrations,django_migrations,2,"CREATE TABLE ""django_migrations"" (""id"" integer..."
1,table,sqlite_sequence,sqlite_sequence,3,"CREATE TABLE sqlite_sequence(name,seq)"
2,table,auth_group,auth_group,9,"CREATE TABLE ""auth_group"" (""id"" integer NOT NU..."
3,index,sqlite_autoindex_auth_group_1,auth_group,10,None
4,table,auth_group_permissions,auth_group_permissions,11,"CREATE TABLE ""auth_group_permissions"" (""id"" in..."
5,table,auth_user_groups,auth_user_groups,15,"CREATE TABLE ""auth_user_groups"" (""id"" integer ..."
6,table,auth_user_user_permissions,auth_user_user_permissions,16,"CREATE TABLE ""auth_user_user_permissions"" (""id..."
7,index,auth_group_permissions_group_id_permission_id_...,auth_group_permissions,20,CREATE UNIQUE INDEX auth_group_permissions_gro...
8,index,auth_group_permissions_group_id_b120cbf9,auth_group_permissions,21,"CREATE INDEX ""auth_group_permissions_group_id_..."
9,index,auth_group_permissions_permission_id_84c5c92e,auth_group_permissions,23,"CREATE INDEX ""auth_group_permissions_permissio..."


In [26]:
df_member = pd.read_sql_query("SELECT * FROM myapp_member", con)
df_room = pd.read_sql_query("SELECT * FROM myapp_room", con)
df_rent = pd.read_sql_query("SELECT * FROM myapp_rent", con, parse_dates=['start','stop'])

In [27]:
df_member.head(1)

,id,name,mobile
0,1,Oscar Williams,


In [28]:
df_room.head(1)

,id,number,hour_price,isavailable
0,6,101,105,1


In [29]:
df_rent.tail(1)

,id,start,stop,member_id,room_id,cost
3426,3809,2018-10-01 18:01:18.462109,NaT,79,10,NaN


In [31]:
df=df_rent.merge(df_member, how='inner', left_on='member_id', right_on='id')\
    .merge(df_room, left_on='room_id', right_on='id')

df.drop(columns=['id_y', 'id', 'mobile'],inplace=True)
df.rename(columns={'id_x':'rent_id'},inplace=True)
df.head()

,rent_id,start,stop,member_id,room_id,cost,name,number,hour_price,isavailable
0,383,2018-05-09 08:01:18.462109,2018-05-09 09:01:18.462109,48,7,100.0,Thomas Thomas,102,100,1
1,1839,2018-07-10 12:01:18.462109,2018-07-10 13:01:18.462109,48,7,100.0,Thomas Thomas,102,100,1
2,2676,2018-08-16 01:01:18.462109,2018-08-16 02:01:18.462109,48,7,100.0,Thomas Thomas,102,100,1
3,3044,2018-08-31 02:01:18.462109,2018-08-31 03:01:18.462109,48,7,100.0,Thomas Thomas,102,100,1
4,3105,2018-09-02 13:01:18.462109,2018-09-02 16:01:18.462109,48,7,300.0,Thomas Thomas,102,100,1


In [32]:
#Q1 Who did spend most money for renting?
df.groupby('name')\
    .agg({'cost':'sum'})\
    .sort_values(by='cost',ascending=False)\
    .head(1)

,cost
name,
Oscar Smith,15116.0


In [33]:
#Q2 Which room does make the most amount of income?

df.groupby('number')\
    .agg({'cost':'sum'})\
    .sort_values(by='cost',ascending=False)\
    .head(1)

,cost
number,
105,269645.0


In [34]:
#Q3 How many time Jack Jones rent the room?
df.query('name=="Jack Jones"').groupby('name').agg({'rent_id':'count'})

,rent_id
name,
Jack Jones,35


In [35]:
#Q4 What are the rooms rented by Claire Taylor?
df.query('name=="Claire Taylor"').groupby('number').agg({'rent_id':'count'})

,rent_id
number,
101,4
102,12
103,4
104,10
105,5


In [42]:
#Q5 what is the total income of ALL rooms in June? 
#Between 1st June 2018(inclusive) and 30th June 2018(inclusive), (use return date).
from datetime import datetime
start = datetime.strptime( "2018-06-01T00:00:00", "%Y-%m-%dT%H:%M:%S" )
stop = datetime.strptime( "2018-06-30T23:59:59", "%Y-%m-%dT%H:%M:%S" )
df[ (start<=df.stop) & (df.stop<stop) ]\
    .groupby('number').agg({'cost':'sum'})

,cost
number,
101,30135.0
102,30400.0
103,48450.0
104,37700.0
105,49551.0


In [75]:
#Q7 Find members that have total rent time exactly 8 hours in June?
df2=df[ (start<=df.stop) & (df.stop<stop) ]
df2['hours']=(df2.stop-df2.start).apply(lambda x: x.seconds//3600)
df2.groupby('name').agg({'hours':'sum'}).query('hours==8')

/home/wasit/gitlab/cs459_django2018/exam/env/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


,hours
name,
James Jones,8
Oscar Johnson,8
William Thomas,8


In [65]:
df

,rent_id,start,stop,member_id,room_id,cost,name,number,hour_price,isavailable,hours,seconds
0,383,2018-05-09 08:01:18.462109,2018-05-09 09:01:18.462109,48,7,100.0,Thomas Thomas,102,100,1,1.0,3600.0
1,1839,2018-07-10 12:01:18.462109,2018-07-10 13:01:18.462109,48,7,100.0,Thomas Thomas,102,100,1,1.0,3600.0
2,2676,2018-08-16 01:01:18.462109,2018-08-16 02:01:18.462109,48,7,100.0,Thomas Thomas,102,100,1,1.0,3600.0
3,3044,2018-08-31 02:01:18.462109,2018-08-31 03:01:18.462109,48,7,100.0,Thomas Thomas,102,100,1,1.0,3600.0
4,3105,2018-09-02 13:01:18.462109,2018-09-02 16:01:18.462109,48,7,300.0,Thomas Thomas,102,100,1,3.0,10800.0
5,3662,2018-09-25 07:01:18.462109,2018-09-25 09:01:18.462109,48,7,200.0,Thomas Thomas,102,100,1,2.0,7200.0
6,388,2018-05-09 13:01:18.462109,2018-05-09 14:01:18.462109,1,7,100.0,Oscar Williams,102,100,1,1.0,3600.0
7,428,2018-05-11 09:01:18.462109,2018-05-11 10:01:18.462109,1,7,100.0,Oscar Williams,102,100,1,1.0,3600.0
8,1397,2018-06-21 13:01:18.462109,2018-06-21 16:01:18.462109,1,7,300.0,Oscar Williams,102,100,1,3.0,10800.0
9,1520,2018-06-27 14:01:18.462109,2018-06-27 18:01:18.462109,1,7,400.0,Oscar Williams,102,100,1,4.0,14400.0


In [63]:
df.iloc[1]

rent_id                              1839
start          2018-07-10 12:01:18.462109
stop           2018-07-10 13:01:18.462109
member_id                              48
room_id                                 7
cost                                  100
name                        Thomas Thomas
number                                102
hour_price                            100
isavailable                             1
hours                                   1
seconds                              3600
Name: 1, dtype: object

In [64]:
df.head(2)

,rent_id,start,stop,member_id,room_id,cost,name,number,hour_price,isavailable,hours,seconds
0,383,2018-05-09 08:01:18.462109,2018-05-09 09:01:18.462109,48,7,100.0,Thomas Thomas,102,100,1,1.0,3600.0
1,1839,2018-07-10 12:01:18.462109,2018-07-10 13:01:18.462109,48,7,100.0,Thomas Thomas,102,100,1,1.0,3600.0
